In [1]:
import os
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import datetime as dt

# set option
pd.set_option('display.max_columns', 999)

# Get data from web

In [2]:
url = 'https://www.worldometers.info/coronavirus'

In [3]:
# get data from url
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
content = soup.find('table', id = 'main_table_countries_yesterday2')

# put data into DataFrame
data = []
attributes = [item.text.replace('\xa0', ' ').replace('\n', '') for item in content.find_all('th')]
for i in content.find_all('tr')[1:]:
    data.append([j.text for j in i.find_all('td')])
df = pd.DataFrame(data, columns = attributes)
df = df[df['#'] != ''].set_index('#')
df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
#,,,,,,,,,,,,,,,,,,,,,
1,China,"205,257","+1,923","4,828",+52,"171,703","+2,323","28,726",341,143,3,"160,000,000","111,163","1,439,323,776",Asia,"7,012","298,120",9,1,0.04,20
2,USA,"82,830,509","+52,013","1,019,463",+356,"80,549,868","+41,054","1,261,178","1,457","247,604","3,047","1,002,337,197","2,996,276","334,527,637",North America,4,328,0,155,1,"3,770"
3,India,"43,065,666","+3,569","523,654",+151,"42,525,563","+2,252","16,449",698,"30,660",373,"835,974,079","595,150","1,404,643,348",Asia,33,"2,682",2,3,0.1,12
4,Brazil,"30,378,061","+22,142","662,941",+164,"29,411,813",,"303,307","8,318","141,094","3,079","63,776,166","296,215","215,303,429",South America,7,325,3,103,0.8,"1,409"
5,France,"28,415,413","+97,498","145,427",+170,"26,245,214","+161,753","2,024,772","1,677","433,586","2,219","266,484,045","4,066,235","65,535,813",Europe,2,451,0,"1,488",3,"30,896"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,MS Zaandam,9,,2,,7,,0,,,,,,,,,,,,,
225,Niue,8,,,,7,,1,,"4,860",,,,"1,646",Australia/Oceania,206,,,,,608
226,Micronesia,3,,,,1,,2,,26,,,,"117,213",Australia/Oceania,"39,071",,,,,17


In [4]:
# export csv file
date = (dt.date.today() + dt.timedelta(days = -2)).strftime('%Y%m%d')
if (os.path.exists('output') == False):
    os.mkdir('output')
df.to_csv('output/Coronavirus_' + date + '.csv', index = False)

# Preprocess

In [5]:
df.dtypes

Country,Other          object
TotalCases             object
NewCases               object
TotalDeaths            object
NewDeaths              object
TotalRecovered         object
NewRecovered           object
ActiveCases            object
Serious,Critical       object
Tot Cases/1M pop       object
Deaths/1M pop          object
TotalTests             object
Tests/1M pop           object
Population             object
Continent              object
1 Caseevery X ppl      object
1 Deathevery X ppl     object
1 Testevery X ppl      object
New Cases/1M pop       object
New Deaths/1M pop      object
Active Cases/1M pop    object
dtype: object

In [6]:
# convert to numeric
df[['NewCases', 'NewDeaths', 'NewRecovered']] = \
    df[['NewCases', 'NewDeaths', 'NewRecovered']].applymap(lambda x: x.replace('+', ''))
df[df.columns.drop(['Country,Other', 'Continent'])] = \
    df[df.columns.drop(['Country,Other', 'Continent'])].applymap(lambda x: x.replace(',', '').replace(' ','')).replace('','0')

# change N/A to nan
df[['Country,Other', 'Continent']] = \
    df[['Country,Other', 'Continent']].replace('','N/A')
df = df.replace('N/A', np.nan)

# change data type
def to_int(s):
    return [int(i) if not(i is np.nan) else np.nan for i in s]
def to_float(s):
    return [float(i) if not(i is np.nan) else np.nan for i in s]
for col in df.columns.drop(['Country,Other', 'Continent']):
    if col in ['New Cases/1M pop', 'New Deaths/1M pop', 'Active Cases/1M pop']:
        df[col] = to_float(df[col])
    else:
        df[col] = to_int(df[col])
        
# fill missing values with mean
for col in df.columns.drop(['Country,Other', 'Continent']):
    df[col].fillna(int(df[col].mean()), inplace = True)
    
# convert some attributes to integer
df['TotalRecovered'] = to_int(df['TotalRecovered'])
df['NewRecovered'] = to_int(df['NewRecovered'])
df['ActiveCases'] = to_int(df['ActiveCases'])

In [7]:
# export csv file
df.to_csv('output/data.csv', index = False)

In [8]:
df.dtypes

Country,Other           object
TotalCases               int64
NewCases                 int64
TotalDeaths              int64
NewDeaths                int64
TotalRecovered           int64
NewRecovered             int64
ActiveCases              int64
Serious,Critical         int64
Tot Cases/1M pop         int64
Deaths/1M pop            int64
TotalTests               int64
Tests/1M pop             int64
Population               int64
Continent               object
1 Caseevery X ppl        int64
1 Deathevery X ppl       int64
1 Testevery X ppl        int64
New Cases/1M pop       float64
New Deaths/1M pop      float64
Active Cases/1M pop    float64
dtype: object

In [9]:
df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
#,,,,,,,,,,,,,,,,,,,,,
1,China,205257,1923,4828,52,171703,2323,28726,341,143,3,160000000,111163,1439323776,Asia,7012,298120,9,1.0,0.04,20.0
2,USA,82830509,52013,1019463,356,80549868,41054,1261178,1457,247604,3047,1002337197,2996276,334527637,North America,4,328,0,155.0,1.00,3770.0
3,India,43065666,3569,523654,151,42525563,2252,16449,698,30660,373,835974079,595150,1404643348,Asia,33,2682,2,3.0,0.10,12.0
4,Brazil,30378061,22142,662941,164,29411813,0,303307,8318,141094,3079,63776166,296215,215303429,South America,7,325,3,103.0,0.80,1409.0
5,France,28415413,97498,145427,170,26245214,161753,2024772,1677,433586,2219,266484045,4066235,65535813,Europe,2,451,0,1488.0,3.00,30896.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,MS Zaandam,9,0,2,0,7,0,0,0,0,0,0,0,0,NaN,0,0,0,0.0,0.00,0.0
225,Niue,8,0,0,0,7,0,1,0,4860,0,0,0,1646,Australia/Oceania,206,0,0,0.0,0.00,608.0
226,Micronesia,3,0,0,0,1,0,2,0,26,0,0,0,117213,Australia/Oceania,39071,0,0,0.0,0.00,17.0
